In [ ]:
# J. Ryu, Electron Microscopy and Spectroscopy Lab., Seoul National University
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.widgets import RectangleSelector
from tabulate import tabulate
from sklearn.cluster import DBSCAN
import tkinter.filedialog as tkf
from scipy import optimize
import ipywidgets as pyw

In [ ]:
# refer to https://scipy-cookbook.readthedocs.io/items/FittingData.html

def gaussian(height, center_x, center_y, width_x, width_y):
    """Returns a gaussian function with the given parameters"""
    width_x = float(width_x)
    width_y = float(width_y)
    return lambda x,y: height*np.exp(
                -(((center_x-x)/width_x)**2+((center_y-y)/width_y)**2)/2)

def moments(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution by calculating its
    moments """
    total = data.sum()
    X, Y = np.indices(data.shape) # row, col
    x = (X*data).sum()/total # row
    y = (Y*data).sum()/total # col
    col = data[:, int(y)]
    width_x = np.sqrt(np.abs((np.arange(col.size)-y)**2*col).sum()/col.sum()) # row
    row = data[int(x), :]
    width_y = np.sqrt(np.abs((np.arange(row.size)-x)**2*row).sum()/row.sum()) # col
    height = data.max()
    return height, x, y, width_x, width_y

def fitgaussian(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution found by a fit"""
    params = moments(data)
    errorfunction = lambda p: np.ravel(gaussian(*p)(*np.indices(data.shape)) -
                                 data)
    p, success = optimize.leastsq(errorfunction, params)
    return p

In [ ]:
def gaussian_center(image, cbox_edge=0):
    y, x = np.indices(image.shape)
    if not cbox_edge:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
        
    else:
        cbox_outy = int(image.shape[0]/2 - cbox_edge/2)
        cbox_outx = int(image.shape[1]/2 - cbox_edge/2)
        center_box = image[cbox_outy:-cbox_outy, cbox_outx:-cbox_outx]
        fit_params = fitgaussian(center_box)
        (_, center_y, center_x, _, _) = fit_params
        center = [center_y+cbox_outy, center_x+cbox_outx]
        
    return center

In [ ]:
def remove_center_beam(image, center=None, cb_rad=0):
    y, x = np.indices(image.shape)
    if not center:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
        
    r = np.hypot(y - center[0], x - center[1])
    r = np.around(r)
    ri = np.where(r<=cb_rad)
    #print(ri[0].shape)
    
    image[ri] = 0
    
    return image

In [ ]:
# refer to github.com/mkolopanis/python/blob/master/radialProfile.py

def radial_variance(image, profile_resolution=1.0, center=None):
   
    y, x = np.indices(image.shape) * profile_resolution
    if not center:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
        
    r = np.hypot(y - center[0]*profile_resolution, x - center[1]*profile_resolution)
    #plt.imshow(r, cmap="Accent")
    #plt.show()

    # Get sorted radii
    ind = np.argsort(r.flat)
    r_sorted = r.flat[ind]
    i_sorted = image.flat[ind]

    # Get the integer part of the radii (bin size = 1)
    r_int = np.around(r_sorted)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = np.where(deltar)[0]       # location of changed radius
    nr = rind[1:] - rind[:-1]        # number of radius bin
    
    csim = np.cumsum(i_sorted, dtype=float)
    sq_csim = np.cumsum(np.square(i_sorted), dtype=float)
    
    #plt.plot(np.square(csim), c="red")
    #plt.plot(sq_csim, c="blue")
    
    avg_square = np.square((csim[rind[1:]] - csim[rind[:-1]]) / nr)
    square_avg = (sq_csim[rind[1:]] - sq_csim[rind[:-1]]) / nr
    
    #plt.plot(avg_square[20:], c="red")
    #plt.plot(square_avg[20:], c="blue")
    
    mask = avg_square.copy()
    mask[np.where(avg_square==0)] = 1.0
    radial_var = (square_avg - avg_square) / mask
    
    return radial_var    

In [ ]:
def load_binary_4D_stack(img_adr, datatype, original_shape, final_shape, log_scale=False):
    stack = np.fromfile(img_adr, dtype=datatype)
    stack = stack.reshape(original_shape)
    print(stack.shape)
    if log_scale:
        stack = np.log(stack[:final_shape[0], :final_shape[1], :final_shape[2], :final_shape[3]])
    else:
        stack = stack[:final_shape[0], :final_shape[1], :final_shape[2], :final_shape[3]]
    
    print(stack.shape) 
    return stack

In [ ]:
raw_adr = tkf.askopenfilename()
print(raw_adr)

In [ ]:
datatype = "float32"
o_shape = (256, 256, 130, 128)
f_shape = (256, 256, 128, 128)

In [ ]:
# load a data
stack_4d = load_binary_4D_stack(raw_adr, datatype, o_shape, f_shape, log_scale=False)
print(np.max(stack_4d))
print(np.min(stack_4d))
print(np.mean(stack_4d))
#print(np.median(stack_4d))

In [ ]:
# select an interesting area
%matplotlib qt
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(np.sum(stack_4d, axis=(2, 3)), cmap="gray")

def onselect(eclick, erelease):
    print('startposition: (%f, %f)' % (eclick.xdata, eclick.ydata))
    print('endposition  : (%f, %f)' % (erelease.xdata, erelease.ydata))

box = RectangleSelector(ax, onselect)
plt.show()

In [ ]:
# crop the data
stack_4d_cropped = stack_4d[int(box.corners[1][0]):int(box.corners[1][2]), 
                                           int(box.corners[0][0]):int(box.corners[0][1])].copy()
print(stack_4d_cropped.shape)
print(np.max(stack_4d_cropped))
print(np.min(stack_4d_cropped))
print(np.mean(stack_4d_cropped))
#print(np.median(stack_4d_cropped))

In [ ]:
%matplotlib inline

In [ ]:
# find center position
center_pos = []
for i in range(stack_4d_cropped.shape[0]):
    for j in range(stack_4d_cropped.shape[1]):
        center_pos.append(gaussian_center(stack_4d_cropped[i, j], cbox_edge=30))
        
center_pos = np.asarray(center_pos)
center_pos = np.reshape(center_pos, (stack_4d_cropped.shape[0], stack_4d_cropped.shape[1], -1))
print(center_pos.shape)
center_mean = np.mean(center_pos, axis=(0, 1))
print(center_mean)

# center distibution
plt.figure()
plt.hist(center_pos[:, :, 0].flatten(), bins=100, density=True, color="orange", label="center y position")
plt.hist(center_pos[:, :, 1].flatten(), bins=100, density=True, color="gray", alpha=0.5, label="center x position")
plt.grid()
plt.legend()
plt.show()

plt.figure()
plt.scatter(center_pos[:, :, 1], center_pos[:, :, 0], s=10.0, alpha=0.5)
plt.grid()
plt.scatter(center_mean[1], center_mean[0], s=20, c="red")
plt.xlabel("center x position", fontsize=20)
plt.ylabel("center y position", fontsize=20)
plt.show()

In [ ]:
# remove center beam
stack_4d_cropped_cb = stack_4d_cropped.copy()
center_radius = 20
for i in range(stack_4d_cropped.shape[0]):
    for j in range(stack_4d_cropped.shape[1]):
        stack_4d_cropped[i, j] = remove_center_beam(stack_4d_cropped[i, j], 
                                                    center=center_mean.tolist(), cb_rad=center_radius)
print(np.max(stack_4d_cropped))
print(np.min(stack_4d_cropped))
print(np.mean(stack_4d_cropped))
#print(np.median(stack_4d_cropped))

In [ ]:
# calculate variance with all angles at a certain k
radial_var_stack = []
len_profile = []
for i in range(stack_4d_cropped.shape[0]):
    for j in range(stack_4d_cropped.shape[1]):
        radial_temp = radial_variance(stack_4d_cropped[i, j], center=center_mean.tolist())
        len_profile.append(len(radial_temp))
        radial_var_stack.append(radial_temp)

if len(np.unique(len_profile)) > 1:
    print(np.unique(len_profile))
    shortest = np.min(len_profile)
    for i in range(len(len_profile)):
        radial_var_stack[i] = radial_var_stack[i][:shortest]

radial_var_stack = np.asarray(radial_var_stack).reshape(stack_4d_cropped.shape[0], stack_4d_cropped.shape[1], -1)
print(radial_var_stack.shape)

radial_var_sum = np.sum(radial_var_stack, axis=(0, 1))
print(radial_var_sum.shape)

plt.figure(figsize=(10,10))
plt.plot(radial_var_sum)
plt.grid()
plt.show()

In [ ]:
%matplotlib inline

In [ ]:
radii = np.arange(len(radial_var_sum))
#pacbed = np.mean(stack_4d_cropped, axis=(0, 1))
pacbed = np.log(np.mean(stack_4d_cropped_cb, axis=(0, 1)))
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(pacbed, cmap="afmhot")
ax.axis("off")
plt.show()

In [ ]:
ki = 27
kf = 29
p1 = 90.0 # lower percentile
p2 = 99.9 # upper percentile
v1 = np.percentile(radial_var_stack[:, :, ki:(kf+1)], p1)
v2 = np.percentile(radial_var_stack[:, :, ki:(kf+1)], p2)
print("lower variance = %f"%v1)
print("upper variance = %f"%v2)

In [ ]:
masked = radial_var_stack[:, :, ki:(kf+1)].copy()
img_tmp = np.ones_like(masked)
img_tmp[np.where(masked < v1)] = 0
img_tmp[np.where(masked > v2)] = 0
img = np.sum(img_tmp, axis=2)
bool_img = np.zeros_like(img)
bool_img[np.where(img != 0)] = 1

fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(img, cmap="afmhot")
ax[0].axis("off")
ax[1].imshow(bool_img, cmap="afmhot")
ax[1].axis("off")
plt.show()

In [ ]:
min_num = 4
epsilon = 2**(1/2)
db = DBSCAN(eps=epsilon, min_samples=min_num)
sel_coor = np.nonzero(bool_img)
X = np.stack((sel_coor[0], sel_coor[1]), axis=1)
db.fit(X)
img_tmp = np.zeros_like(bool_img)
img_tmp += -1
img_tmp[X[:, 0], X[:, 1]] = db.labels_
labeled_cluster_img = img_tmp.copy()
cluster_img = np.where(img_tmp<0, img_tmp, 1)
cluster_img = np.where(cluster_img>-1, bool_img, 0)

label = np.unique(db.labels_)[1:]
print("number of clusters = %d"%(len(label)))
print(label)

fig, ax = plt.subplots(1, 3, figsize=(10, 10))
ax[0].imshow(img, cmap="afmhot")
ax[0].axis("off")
ax[1].imshow(bool_img, cmap="afmhot")
ax[1].axis("off")
ax[2].imshow(cluster_img, cmap="afmhot")
ax[2].axis("off")

for i in label:
    l_coor = np.where(db.labels_ == i)
    ax[2].text(X[l_coor[0][0], 1], X[l_coor[0][0], 0], str(i), 
                      color="red", fontdict={"size": 10})

fig.tight_layout()
plt.show()

In [ ]:
def radial_indices(shape, radial_range, center=None):
    y, x = np.indices(shape)
    if not center:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
    
    r = np.hypot(y - center[0], x - center[1])
    ri = np.ones(r.shape)
    
    if len(np.unique(radial_range)) > 1:
        ri[np.where(r <= radial_range[0])] = 0
        ri[np.where(r > radial_range[1])] = 0
        
    else:
        r = np.round(r)
        ri[np.where(r != round(radial_range[0]))] = 0
    
    return ri

In [ ]:
def indices_at_r(shape, radius, center=None):
    y, x = np.indices(shape)
    if not center:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
    r = np.hypot(y - center[0], x - center[1])
    r = np.around(r)
    
    ri = np.where(r == radius)
    
    angle_arr = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            angle_arr[i, j] = np.angle(complex(x[i, j]-center[1], y[i, j]-center[0]), deg=True)
            
    angle_arr = angle_arr + 180
    angle_arr = np.around(angle_arr)
    
    ai = np.argsort(angle_arr[ri])
    r_sort = (ri[0][ai], ri[1][ai])
    a_sort = np.sort(angle_arr[ri])
        
    return r_sort, a_sort

In [ ]:
%matplotlib qt
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111)
cimg = ax.imshow(img, cmap="afmhot", picker=True)
cimg.axes.set_title("click a pixel")
cimg.axes.axis("off")
print(cimg)

kr = np.around((kf + ki)/2)
k_range = np.arange(kr-3, kf+3, 1)
print(k_range)
num_k = len(k_range)
k_donut = radial_indices(f_shape[2:], [k_range[0], k_range[-1]], center=center_mean.tolist())

def onpick(event):
    
    if event.artist != cimg: return True

    if event.mouseevent.name != "button_press_event": return True

    sy, sx = int(event.mouseevent.ydata), int(event.mouseevent.xdata)
    #print(sy, sx)
    
    figi = plt.figure(figsize=(15, 10))
    G = gridspec.GridSpec(num_k, num_k*4)
    ax1 = figi.add_subplot(G[:num_k, :num_k])
    ax2 = figi.add_subplot(G[:num_k, num_k:num_k*2])
    ax3 = figi.add_subplot(G[:num_k, num_k*2:num_k*3])

    ax1.imshow(img, cmap="afmhot", picker=False)
    ax1.scatter(sx, sy, marker="s", facecolor="none", edgecolors="aqua", linewidths=1.5)
    ax1.axis("off")
    
    ax2.imshow(stack_4d_cropped[sy, sx], cmap="afmhot")
    ax2.axis("off")
    
    ax3.imshow(np.log(stack_4d_cropped_cb[sy, sx]), cmap="afmhot")
    ax3.imshow(k_donut, cmap="gray", alpha=0.20)
    ax3.axis("off")
    
    for i in range(num_k):
        ax_tmp = figi.add_subplot(G[i, num_k*3:num_k*4])
        k_ind, a_ind = indices_at_r(f_shape[2:], radius=k_range[-(i+1)], center=center_mean.tolist())
        k_line = stack_4d_cropped_cb[int(sy), int(sx)][k_ind]
        ax_tmp.plot(a_ind, k_line, linewidth=1)
        ax_tmp.grid()
        ax_tmp.set_yticks([])
        ax_tmp.tick_params(axis="x", labelsize=5)
    
    figi.canvas.draw()
    figi.tight_layout()
    
    return True

fig.canvas.mpl_connect('pick_event', onpick)
fig.tight_layout()

plt.show()

In [ ]:
%matplotlib qt
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111)
ximg = ax.imshow(cluster_img, cmap="afmhot", picker=True)
for i in label:
    l_coor = np.where(db.labels_ == i)
    ximg.axes.text(X[l_coor[0][0], 1], X[l_coor[0][0], 0], str(i), 
                      color="red", fontdict={"size": 10})
ximg.axes.set_title("click a pixel")
ximg.axes.axis("off")
print(ximg)

k_fix = 27

def onpick(event):
    
    if event.artist != ximg: return True

    if event.mouseevent.name != "button_press_event": return True

    sy, sx = int(event.mouseevent.ydata), int(event.mouseevent.xdata)
    #print(sy, sx)
    
    label_id = labeled_cluster_img[sy, sx]
    print(label_id)
        
    if label_id == -1:
        num_p = 9
        mask = np.zeros(img.shape)
        mask[(sy-1):(sy+2), (sx-1):(sx+2)] = 100
        sel_ind = np.where(mask == 100)
        
    else:
        sel_ind = np.where(labeled_cluster_img == label_id)
        mask = np.zeros(img.shape)
        mask[sel_ind] = 100
        num_p = len(sel_ind[0])
    
    print(num_p)
    figi = plt.figure(figsize=(15, 10))
    G = gridspec.GridSpec(num_p, num_p*3)
    ax1 = figi.add_subplot(G[:num_p, :num_p])
    ax2 = figi.add_subplot(G[:num_p, num_p:num_p*2])

    ax1.imshow(cluster_img, cmap="afmhot")
    for i in label:
        l_coor = np.where(db.labels_ == i)
        ax1.text(X[l_coor[0][0], 1], X[l_coor[0][0], 0], str(i), 
                      color="red", fontdict={"size": 10})
    ax1.axis("off")
    
    ax2.imshow(mask, cmap="Reds")
    ax2.imshow(cluster_img, cmap="gray", alpha=0.1)
    ax2.axis("off")
    
    k_ind, a_ind = indices_at_r(f_shape[2:], radius=k_fix, center=center_mean.tolist())
    for i in range(num_p):
        ax_tmp = figi.add_subplot(G[i, num_p*2:num_p*3])
        k_line = stack_4d_cropped_cb[sel_ind[0][i], sel_ind[1][i]][k_ind]
        ax_tmp.plot(a_ind, k_line, linewidth=1)
        ax_tmp.grid()
        ax_tmp.set_yticks([])
        ax_tmp.tick_params(axis="x", labelsize=5)
    
    figi.canvas.draw()
    figi.tight_layout()
    
    return True

fig.canvas.mpl_connect('pick_event', onpick)
fig.tight_layout()

plt.show()

In [ ]:
low_var_coor = np.where(img == 0)
print(low_var_coor[0].shape)
row = 2
col = 2
rand_int = np.random.randint(0, low_var_coor[0].shape, row * col)
print(rand_int)

fig, ax = plt.subplots(row, col, figsize=(10, 10))
ax_flat = ax.flat
for i, a in enumerate(ax_flat):
    a.imshow(np.log(stack_4d_cropped_cb[low_var_coor[0][rand_int[i]], low_var_coor[1][rand_int[i]]]), cmap="afmhot")
    a.axis("off")
fig.tight_layout()
plt.show()

In [ ]:
sel_label = 27

cluster_coor = np.where(labeled_cluster_img == sel_label)
print(cluster_coor[0].shape)
row = 2
col = 2
print(row * col)
fig, ax = plt.subplots(row, col, figsize=(10, 10))
ax_flat = ax.flat

num_dp = row * col
if num_dp > cluster_coor[0].shape[0]:
    num_dp = cluster_coor[0].shape[0]

print(num_dp)
for i in range(num_dp):
    ax_flat[i].imshow(np.log(stack_4d_cropped_cb[cluster_coor[0][i], cluster_coor[1][i]]), cmap="afmhot")
    ax_flat[i].axis("off")
fig.tight_layout()
plt.show()